# AFSK Demodulator
## Step 2: Band-Pass FIR Filter

This is a Pynq portion of the AFSK demodulator project.  We will be using the FPGA overlay that we created in Vivado.

At this point we have created the bitstream for "project_02" and copied the bitstream, TCL wrapper, and hardware hand-off file to the Pynq board.

Let's first verify that we can load the module.

In [9]:
from pynq import Overlay, Xlnk
import numpy as np
import pynq.lib.dma

overlay = Overlay('project_02.bit')
dma = overlay.bpfilter.bpf_dma

## Accellerating FIR Filter

Below is the implementation of the AFSK demodulator in Python.  We are going to remove the band pass filter code and replace it with new code.

In [ ]:
import sys
sys.path.append('../../base')

import numpy as np
from scipy.signal import lfiltic, lfilter, firwin
from scipy.io.wavfile import read
from DigitalPLL import DigitalPLL
from HDLC import HDLC
from AX25 import AX25
import time

from pynq import Overlay, Xlnk
import numpy as np

block_size = 1024*1024

xlnk = Xlnk()
out_buffer = xlnk.cma_array(shape=(block_size,), dtype=np.int16)
in_buffer = xlnk.cma_array(shape=(block_size,), dtype=np.int16)

def bpf(data):
    start_time = time.time()
    output = np.array([],dtype=np.int16)
    for i in range(0, len(data), block_size):
        print(i)
        out_buffer[:len(data[i:i+block_size])] = data[i:i+block_size]
        dma.sendchannel.transfer(out_buffer)
        dma.recvchannel.transfer(in_buffer)
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        output = np.append(output, in_buffer)
    stop_time = time.time()
    sw_exec_time = stop_time - start_time
    print('Hardware FIR execution time: ',sw_exec_time)
    return output

class fir_filter(object):
    def __init__(self, coeffs):
        self.coeffs = coeffs
        self.zl = lfiltic(self.coeffs, 32768.0, [], [])
    def __call__(self, data):
        result, self.zl = lfilter(self.coeffs, 32768.0, data, -1, self.zl)
        return result

class NRZI:

    def __init__(self):

        self.state = False

    def __call__(self, x):
        
        result = (x == self.state)
        self.state = x
        return result


audio_file = read('../../base/TNC_Test_Ver-1.101-26400-1min.wav')
sample_rate = audio_file[0]
audio_data = audio_file[1]
delay = 12 # ~446us

lpf_coeffs = np.array(firwin(101, [1200.0/(sample_rate/2)], width = None,
        pass_zero = True, scale = True, window='hann') * 32768, dtype=int)

lpf = fir_filter(lpf_coeffs)

filter_delay = 64 + 50

# Band-pass filter the audio data
fa = bpf(audio_data)
fz = bpf(np.zeros(filter_delay))
print("Appending data...")
f = np.append(fa, fz)[filter_delay:]
# Digitize the data
print("Digitizing audio data...")
d = np.greater_equal(f, 0, dtype=int)
# Delay the data
print("Delay...")
a = d[delay:]
# XOR the digitized data with the delayed version
print("Doing Logical XOR...")
x = np.logical_xor(d[:0-delay], a, dtype=int)
# Low-pass filter the PWM signal
print("Doing LPF...")
c = lpf(x-0.5)
# Digitize the tone transistions
print("Digitizing correlator output...")
dx = np.greater_equal(c, 0.0)
# Create the PLL
pll = DigitalPLL(sample_rate, 1200.0)

locked = np.zeros(len(dx), dtype=int)
sample = np.zeros(len(dx), dtype=int)

# Clock recovery
print("Doing clock recovery...")
for i in range(len(dx)):
    sample[i] = pll(dx[i])
    locked[i] = pll.locked()
    
nrzi = NRZI()

print("Doing NRZI...")
data = [int(nrzi(x)) for x,y in zip(dx, sample) if y]

hdlc = HDLC()

print("Doing HDLC")
count = 0
for b,s,l in zip(dx, sample, locked):
    if s:
        packet = hdlc(nrzi(b), l)
        if packet is not None:
            count += 1
            print(count, AX25(packet[1]))


0
1048576
Hardware FIR execution time:  0.2691988945007324
0
Hardware FIR execution time:  0.11102795600891113
Appending data...
Digitizing audio data...
Delay...
Doing Logical XOR...
Doing LPF...


In [ ]:
ctrl = dma.recvchannel._mmio.read(dma.recvchannel._offset)
print(ctrl)
dma.recvchannel._mmio.write(dma.recvchannel._offset, (ctrl | 4) & 0xFFFFFFFE)
print(dma.recvchannel._mmio.read(dma.recvchannel._offset+0x04))
dma.recvchannel.start()
dma.sendchannel.start()

In [ ]:
xlnk.xlnk_reset()